In [74]:
import pandas as pd
df = pd.read_csv('housing.csv')
df.info()
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')


In [75]:
# creacion del train y del test

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state= 22)

train.to_csv("housing_train.csv", index = False)
test.to_csv("housing_test.csv", index = False)


In [76]:
# correlación de las variables
# df_for_correlation = df.drop(['ocean_proximity'], axis = 1)
# # df_for_correlation.info()
# correlation =  df_for_correlation.corr()
# correlation['median_house_value'].sort_values(ascending=False)
# df.corr()

#df['median_house_value'].sort_values(ascending= True)


In [77]:
#seseparamos características a entrenar del target
df_train = pd.read_csv("housing_train.csv")
data, labels = df_train.drop(['median_house_value'], axis=1), df_train['median_house_value'].copy()
data.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-119.72,36.76,23.0,6403.0,NaN,3573.0,1260.0,2.3006,INLAND
1,-120.79,38.70,13.0,5036.0,1034.0,2243.0,923.0,2.3319,INLAND
2,-118.20,34.04,18.0,796.0,227.0,547.0,218.0,1.0333,<1H OCEAN
3,-117.34,33.21,12.0,5963.0,1372.0,3015.0,1124.0,2.7386,NEAR OCEAN
4,-121.46,38.54,48.0,1001.0,205.0,605.0,175.0,1.8333,INLAND


Variables numéricas


In [78]:
#nos quedamos con las variables numericas
print(data.columns)

data_num = data.drop(['ocean_proximity'], axis = 1)
data_num.head()

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-119.72,36.76,23.0,6403.0,NaN,3573.0,1260.0,2.3006
1,-120.79,38.70,13.0,5036.0,1034.0,2243.0,923.0,2.3319
2,-118.20,34.04,18.0,796.0,227.0,547.0,218.0,1.0333
3,-117.34,33.21,12.0,5963.0,1372.0,3015.0,1124.0,2.7386
4,-121.46,38.54,48.0,1001.0,205.0,605.0,175.0,1.8333


In [79]:
#nos quedamos con las no numericas

data_no_num = data[['ocean_proximity']]
data_no_num.head()

,ocean_proximity
0,INLAND
1,INLAND
2,<1H OCEAN
3,NEAR OCEAN
4,INLAND


In [80]:
#ver missing values del data set

data_num.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        163
population              0
households              0
median_income           0
dtype: int64

In [81]:
#rellenar missing values con la mediana de los demás

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
imputer.fit(data_num)
imputer.statistics_

array([-118.51  ,   34.26  ,   29.    , 2126.    ,  435.    , 1165.    ,
        410.    ,    3.5334])

In [82]:
#Rellenar el dataset
X = imputer.transform(data_num)
data_filled = pd.DataFrame(X, columns=data_num.columns, index = data.index)
#comprobar si ahora no es null
data_filled.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

In [83]:
#los sistemas de ML solo son capaces de trabajar con datos 
#numericos asi q tranformamos los no numericos en numericos 
#mediante el uso de OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
no_num_encoder = OneHotEncoder()
data_no_num_encoded = no_num_encoder.fit_transform(data_no_num)
data_no_num_encoded_df=pd.DataFrame(data_no_num_encoded.toarray()) 
print(type(data_no_num_encoded_df))

<class 'pandas.core.frame.DataFrame'>


In [84]:
#ahora normalizamos los datos numericos usando un StandardScaler

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_num_scaled = scaler.fit_transform(data_num)
data_tr = pd.DataFrame(data_num_scaled, columns=data_num.columns, index=data.index)
data_tr.describe()




,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.634900e+04,1.651200e+04,1.651200e+04,1.651200e+04
mean,7.217525e-16,6.491362e-16,1.207583e-16,2.216143e-17,-2.933613e-18,7.627404e-17,4.028862e-17,-6.981926e-17
std,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000031e+00,1.000030e+00,1.000030e+00,1.000030e+00
min,-2.381604e+00,-1.445235e+00,-2.201136e+00,-1.214416e+00,-1.278351e+00,-1.256377e+00,-1.310671e+00,-1.764526e+00
25%,-1.105456e+00,-7.998537e-01,-8.491991e-01,-5.457575e-01,-5.776820e-01,-5.636872e-01,-5.758077e-01,-6.848289e-01
50%,5.324548e-01,-6.455233e-01,2.558386e-02,-2.329159e-01,-2.440301e-01,-2.270859e-01,-2.333984e-01,-1.765975e-01
75%,7.819462e-01,9.679302e-01,6.617896e-01,2.338048e-01,2.564478e-01,2.609859e-01,2.723138e-01,4.571083e-01
max,2.628182e+00,2.950841e+00,1.854675e+00,1.631534e+01,1.407917e+01,3.034782e+01,1.470618e+01,5.825810e+00


In [96]:
#concatenar info numerica con no numerica

result = pd.concat([data_tr, data_no_num_encoded_df], axis = 1)
result.shape()

TypeError: 'tuple' object is not callable